In [1]:
using Revise

using RigidBodyDynamics
using RigidBodyDynamics: Bounds

using Plots
using BilevelTrajOpt

In [2]:
urdf = joinpath("..", "urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)
body = findbody(mechanism, "ball")
basejoint = joint_to_parent(body, mechanism)
floatingjoint = Joint(basejoint.name, frame_before(basejoint), frame_after(basejoint), QuaternionFloating{Float64}())
replace_joint!(mechanism, basejoint, floatingjoint)
position_bounds(floatingjoint) .= Bounds(-100, 100)
velocity_bounds(floatingjoint) .= Bounds(-100, 100)
μ = 0.5
motion_type = :xyz
point = Point3D(default_frame(findbody(mechanism, "floor")), [0.,0.,0.])
normal = FreeVector3D(default_frame(findbody(mechanism, "floor")), [0.,0.,1.])
hs = HalfSpace(point, normal)
floor = Obstacle(hs, μ, motion_type)
obstacles = [floor]
env = parse_contacts(mechanism, urdf, obstacles)
x0 = MechanismState(mechanism)
Δt = 0.005
sim_data = get_sim_data(x0,env,Δt,true);

In [3]:
q0 = [1., 0., 0., 0., 0., 0., 0.]
v0 = [0., 0., 0., 0.5, 0., 0.]

u0 = zeros(sim_data.num_v)
set_configuration!(x0, q0)
set_velocity!(x0, v0)
setdirty!(x0)
ctrl! = (u,t,x) -> u[:] = 0.
traj = BilevelTrajOpt.simulate(x0,env,sim_data.Δt,1,ctrl!,implicit_contact=false)
qnext = traj[1:sim_data.num_q,2]
vnext = traj[sim_data.num_q+1:sim_data.num_q+sim_data.num_v,2]
x_sol_exp = traj[sim_data.num_q+sim_data.num_v+2:end,2]

# qnext = q0
# vnext = v0

τ_ip, x_sol_ip, λ_ip, μ_ip, fx_ip = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=true);

τ_auglag, x_sol_auglag, λ_auglag, μ_auglag, fx_auglag = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=false);

# display("Explicit")
# display(x_sol_exp)
display("IP")
display(x_sol_ip)
display("Aug Lag")
display(x_sol_auglag)

display("torques")
display(τ_ip)
display(τ_auglag)

elapsed time: 22.959399816 seconds
elapsed time: 0.004501159 seconds

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

elapsed time: 0.004152719 seconds
elapsed time: 0.004037703 seconds
elapsed time: 0.004354215 seconds
elapsed time: 0.004277462 seconds
elapsed time: 0.004182745 seconds
elapsed time: 0.004089139 seconds
elapsed time: 0.004229572 seconds
elapsed time: 0.004169459 seconds
elapsed time: 0.004175872 seconds
elapsed time: 0.013252909 seconds
elapsed time: 0.004085363 seconds
elapsed time: 0.004824823 seconds
elapsed time: 0.003827821 seconds
elapsed time: 0.003756678 seconds
elapsed time: 0.003770153 seconds
Solve_Succeeded
Solve_Succeeded

"IP"

6-element Array{Float64,1}:
 -9.65697e-9
 -9.80573e-9
  0.500003  
 -9.80573e-9
  0.475475  
  1.00001   

"Aug Lag"

6-element Array{Float64,1}:
  0.00322556
 -0.00322556
  0.503229  
 -0.00322556
  0.47548   
  1.00001   

"torques"

6-element Array{Float64,1}:
  0.0        
  0.0        
  0.0        
  4.90503    
 -6.00693e-16
 -9.81006    

6-element Array{Float64,1}:
  0.0        
  0.0        
  0.0        
  4.90503    
 -1.51559e-10
 -9.81006    

In [ ]:
J_auto = ForwardDiff.jacobian(qv -> solve_implicit_contact_τ(sim_data,q0,v0,u0,qv[1:sim_data.num_q],qv[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=false)[2], vcat(qnext,vnext))

In [4]:
τ_auglag, x_sol_auglag, λ_auglag, μ_auglag, fx_auglag = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=false);

# autodiff 
J_auto = ForwardDiff.jacobian(qv -> solve_implicit_contact_τ(sim_data,q0,v0,u0,qv[1:sim_data.num_q],qv[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=false)[2], vcat(qnext,vnext))
Jt_auto = ForwardDiff.jacobian(qv -> solve_implicit_contact_τ(sim_data,q0,v0,u0,qv[1:sim_data.num_q],qv[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=false)[1], vcat(qnext,vnext))

# numerical
ϵ = 1e-8
J_num = zeros(size(J_auto))
Jt_num = zeros(size(J_auto))
# Jt_num_ip = zeros(size(J_auto))
qv = vcat(qnext,vnext)
for i = 1:length(qv)
    δ = zeros(length(qv))
    δ[i] = ϵ 
    τ, x, λ, μ, fx = solve_implicit_contact_τ(sim_data,q0,v0,u0,(qv+δ)[1:sim_data.num_q],(qv+δ)[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=false)
    J_num[:,i] = (x - x_sol_auglag)/ϵ
    Jt_num[:,i] = (τ - τ_auglag)/ϵ
#     τ, x, λ, μ, fx = solve_implicit_contact_τ(sim_data,q0,v0,u0,(qv+δ)[1:sim_data.num_q],(qv+δ)[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=true)
#     Jt_num_ip[:,i] = (τ - τ_auglag)/ϵ
end

display(J_auto)
display(J_num)

display(Jt_auto)
display(Jt_num)
# display(Jt_num_ip)

6×13 Array{Float64,2}:
 0.0   6.19669e-10  0.0650667    3.43696e-7  …   -7.13768e-6   1.3265     
 0.0  -6.37861e-6   0.184941    -0.500002        10.1937       3.77033    
 0.0   5.94195e-10  0.065054    -1.64804e-6       3.36404e-5   1.3265     
 0.0   6.38204e-6   0.184942     0.500004       -10.1937       3.77035    
 0.0   4.66832e-11  8.46594e-6   4.2604e-11      -1.12142e-9   0.000159928
 0.0   9.28221e-9   1.00001      1.03252e-6  …   -2.10535e-5  20.3874     

6×13 Array{Float64,2}:
 0.0   0.000886324   0.0689028     0.00155451  …    0.0068732     1.32941   
 0.0  -0.00764826    0.178803     -0.511293        10.1982        3.75937   
 0.0   0.0180631     0.0670094     0.0173473       -0.00693997    1.3308    
 0.0  -0.00497627    0.186535      0.491224       -10.1903        3.77905   
 0.0   0.000891509  -0.000243661  -0.00028999       0.000132871   0.00039726
 0.0   0.0111475     0.999766     -0.00766291  …    0.00987892   20.3941    

6×13 Array{Float64,2}:
 0.0    0.0           0.0         …     0.0             0.0        
 0.0    0.0           0.0               0.0             0.0        
 0.0    6.93889e-18   0.0               0.0             0.0        
 0.0   -2.49898e-10  19.62              0.000400033     5.99176e-6 
 0.0  -19.62          6.97513e-6     -200.0             0.000142201
 0.0   -9.08016e-8    1.56365e-6  …     0.000206535  -200.0        

6×13 Array{Float64,2}:
 0.0    0.0           0.0          0.0          …     0.0           0.0      
 0.0    0.0           0.0          3.46945e-10        0.0           0.0      
 0.0    1.03398e-17   2.01948e-20  0.0                0.0           0.0      
 0.0    0.168504     19.6016       0.154928          -0.135507      0.0136613
 0.0  -19.5939        0.07585      0.0246288       -200.011         0.193045 
 0.0   -0.109357      0.00235794   0.0751731    …    -0.0969122  -200.066    

In [ ]:
display(Jt_auto[:,11:13])
display(Jt_num[:,11:13])
display(Jt_num_ip[:,11:13])

In [ ]:
 (J_num[:,1] - x_sol_auglag)/ϵ

In [5]:
maximum(abs.(J_auto-J_num))

0.03702111061622509

In [ ]:
abs.(J_auto-J_num)

In [ ]:
J_num

In [ ]:
J_auto